# COGS 108 - Data Checkpoint

# Names

- Jake Heinlein
- Nathan Tripp
- Naomi Chin
- Leo Friedman
- Dante Tanjuatco

<a id='research_question'></a>
# Research Question

*Is the combination of an mlb free agents age and batting performance, measured by batting average and on base plus slugging percentage, indicative of their yearly salary, and if so, can an algorithm predict a players future contract based on these factors?*

# Dataset(s)

Dataset 1: 
- Dataset name: MLB Free Agent Contracts 1991-2023
- Link: https://docs.google.com/spreadsheets/d/1bXUPBabVf82y0m2KaZ0F9Fno9xwZ2pmepbFvMBX_TEM/edit#gid=1265430999
- Number of observations: 4995
- Description: Dataset of all MLB Free Agent Contracts given from 1991-2023. Each observation contains relevant variables such salary, year signed, player name, and player age. 

Dataset 2: 
- Dataset name: MLB Batting Stats 1871-2022
- Link to dataset: https://github.com/chadwickbureau/baseballdatabank/blob/master/core/Batting.csv
- Number of observations: 110458
- Description: Dataset of all MLB player batting statistics from from 1871-2022. Each observation contains variables (hits, walks, doubles, singles, homeruns, etc.) that can be used to calculate batting average and on base percentace plus slugging average. 

Dataset 3: 
- Dataset name: MLB Batting Player Information 1871-2022
- Link to the dataset: https://github.com/chadwickbureau/baseballdatabank/blob/master/core/People.csv
- Number of observations: 20369
- Description: Dataset of general information about MLB players from 1871-2022. Each observation contains variables relating to the player such as age, name, and birthday. 

Combining datasets: 
- We plan to combine dataset 2 and 3 by adding the player names in dataset 3 to a column in dataset 2. This is possible because dataset 2 and 3 each share the variable playerIDs. We plan on comparing values in dataset 1 with values in dataset 2 based on player names. We will achieve this by standardizing each player's name. 

# Setup

In [1018]:
# import packages and setup visuals
import pandas as pd
import os

In [1019]:
# merge all contract files in contracts directory as one dataframe
directory = 'data/contracts/'
filepaths = [directory + filename for filename in os.listdir(directory)]
contracts = pd.concat([pd.read_csv(filepath) for filepath in filepaths])
print('contracts shape: ', contracts.shape)

# read people and batting files as dataframes
people = pd.read_csv('data/batting/People.csv')
batting = pd.read_csv('data/batting/Batting.csv')
print('people shape: ', people.shape)
print('batting shape: ', batting.shape)

contracts shape:  (4996, 18)
people shape:  (20370, 24)
batting shape:  (110495, 22)


# Data Cleaning

##### Part 1: Cleaning contacts dataframe

In [1020]:
# drop unecessary columns
contracts = contracts[['Player','Pos\'n', 'Age', 'Term', 'AAV']]

# drop players that are pitchers or have NaN values
contracts = contracts[contracts["Pos'n"].str.contains("hp") == False]
contracts = contracts.dropna(axis=0)

# drop position column (no longer needed)
contracts = contracts.drop('Pos\'n',axis=1)
contracts

,Player,Age,Term,AAV
0,"Cano, Robinson",31.0,2014-23,"$24,000,000"
1,"Ellsbury, Jacoby",30.0,2014-20,"$21,857,143"
2,"Choo, Shin-Soo",31.0,2014-20,"$18,571,429"
3,"McCann, Brian",30.0,2014-18,"$17,000,000"
4,"Granderson, Curtis",33.0,2014-17,"$15,000,000"
...,...,...,...,...
66,"Chisenhall, Lonnie",30.0,2019,"$2,750,000"
70,"Maldonado, Martin",32.0,2019,"$2,500,000"
75,"McCann, Brian",35.0,2019,"$2,000,000"
77,"Harrison, Josh",31.0,2019,"$2,000,000"


In [1021]:
# functions to standardize variables
def salary_to_int(str_in):
    return int(str_in.replace('$','').replace(',',''))

def term_to_year(str_in):
    return int(str(str_in).split('-')[0])

In [1022]:
# standardize salary and term variables
contracts['AAV'] = contracts['AAV'].apply(salary_to_int)
contracts['Term'] = contracts['Term'].apply(term_to_year)
contracts

,Player,Age,Term,AAV
0,"Cano, Robinson",31.0,2014,24000000
1,"Ellsbury, Jacoby",30.0,2014,21857143
2,"Choo, Shin-Soo",31.0,2014,18571429
3,"McCann, Brian",30.0,2014,17000000
4,"Granderson, Curtis",33.0,2014,15000000
...,...,...,...,...
66,"Chisenhall, Lonnie",30.0,2019,2750000
70,"Maldonado, Martin",32.0,2019,2500000
75,"McCann, Brian",35.0,2019,2000000
77,"Harrison, Josh",31.0,2019,2000000


In [1023]:
# rename columns for consistency
contracts.columns = ['playerName','playerAge','year','yearSalary']
contracts.head()

,playerName,playerAge,year,yearSalary
0,"Cano, Robinson",31.0,2014,24000000
1,"Ellsbury, Jacoby",30.0,2014,21857143
2,"Choo, Shin-Soo",31.0,2014,18571429
3,"McCann, Brian",30.0,2014,17000000
4,"Granderson, Curtis",33.0,2014,15000000


##### Part 2: Clean batting dataframe 

In [1024]:
# functions used for calculating batting average and OBS average
def calc_avg(h, ab):
    return h / ab
    
def calc_obp(h, bb, hbp, ab, sf):
    return (h + bb + hbp) / (ab + bb + sf + hbp)
    
def calc_tb(h, two_b, three_b, hr):
    singles = h - two_b - three_b - hr
    return singles + two_b * 2 + three_b * 3 + hr * 4

def calc_slg(tb, ab):
    return tb / ab

def calc_obs(obp, slg):
    return obp + slg

In [1025]:
# create batting average column in batting dataframe
avg = calc_avg(h=batting['H'], ab=batting['AB'])
batting['AVG'] = round(avg, 3)

# create OBS average column in batting dataframe
obp = calc_obp(h=batting['H'], bb=batting['BB'], hbp=batting['HBP'], ab=batting['AB'], sf=batting['SF'])
tb = calc_tb(h=batting['H'], two_b=batting['2B'], three_b=batting['3B'], hr=batting['HR'])
slg = calc_slg(tb, batting['AB'])
obs = calc_obs(obp, slg)
batting['OBS'] = round(obs, 3)
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,CS,BB,SO,IBB,HBP,SH,SF,GIDP,AVG,OBS
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.000,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0,0.271,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0,0.292,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0,0.331,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0,0.325,NaN


In [1026]:
# drop irrelevant columns and rename
batting = batting[['playerID', 'yearID', 'teamID', 'AVG', 'OBS']]
batting.columns = ['playerID', 'year', 'team', 'AVG', 'OBS']

# drop observations with NaN values
batting = batting.dropna(axis=0)
batting.head()

,playerID,year,team,AVG,OBS
36887,aaronha01,1954,ML1,0.280,0.769
36888,aberal01,1954,DET,0.128,0.295
36889,abramca01,1954,PIT,0.143,0.522
36890,abramca01,1954,BAL,0.293,0.821
36891,adamsbo03,1954,CIN,0.269,0.749


In [1027]:
# add name column to people that combines players first and last name
people['name'] = people['nameFirst'] + ' ' + people['nameLast']

# drop irrelevant columns
people = people[['playerID','name']]

# merge batting and people to add name column to batting dataframe
batting = batting.merge(people, how='left', on='playerID')

# rename and reorder  batting columns for consistency 
batting = batting.rename({'name':'playerName'}, axis=1)
batting = batting[['playerID','playerName','team','year','AVG','OBS']]
batting.head()

,playerID,playerName,team,year,AVG,OBS
0,aaronha01,Hank Aaron,ML1,1954,0.280,0.769
1,aberal01,Al Aber,DET,1954,0.128,0.295
2,abramca01,Cal Abrams,PIT,1954,0.143,0.522
3,abramca01,Cal Abrams,BAL,1954,0.293,0.821
4,adamsbo03,Bobby Adams,CIN,1954,0.269,0.749
